In [24]:
# 去重,建立字符到索引的映射idx_to_char
idx_to_char = list(set(corpus_chars))
# 建立索引到字符的映射corpus_indices
char_to_idx = {idx:char for char, idx in enumerate(idx_to_char)}
vocab_size = len(char_to_idx)
print(vocab_size)

corpus_indices = [char_to_idx[char]  for char in corpus_chars]
sample = corpus_indices[:20]
print('chars:', ''.join([idx_to_char[idx] for idx in sample]))
print('indices:', sample)

## 时间序列的采样
对于时间序列的采样，进行的操作是对于一句话，选取一定的时间步数，之后再句子中选取连续的时间步数的词组char作为训练集的样本（也就是下面的X），而其对应的char向后退移动一步作为训练样本对应的label（也就是下面的Y）。
举个例子：
现在我们考虑序列“想要有直升机，想要和你飞到宇宙去”，如果时间步数为5，有以下可能的样本和标签：
* $X$：“想要有直升”，$Y$：“要有直升机”
* $X$：“要有直升机”，$Y$：“有直升机，”
* $X$：“有直升机，”，$Y$：“直升机，想”
* ...
* $X$：“要和你飞到”，$Y$：“和你飞到宇”
* $X$：“和你飞到宇”，$Y$：“你飞到宇宙”
* $X$：“你飞到宇宙”，$Y$：“飞到宇宙去”
可以看到，如果序列的长度为T，时间步数为n，那么一共有T-n个合法的样本，但是这些样本有大量的重合，我们通常采用更加高效的采样方式。我们有两种方式对时序数据进行采样，分别是随机采样和相邻采样。

## 随机采样
下面的代码每次从数据里随机采样一个小批量。其中批量大小`batch_size`是每个小批量的样本数，`num_steps`是每个样本所包含的时间步数。
在随机采样中，每个样本是原始序列上任意截取的一段序列，相邻的两个随机小批量在原始序列上的位置不一定相毗邻。**即先切片再随机抽样。**

In [29]:
import torch
import random
def data_iter_random(corpus_indices, batch_size, num_steps, device=None):
    # 减1是因为对于长度为n的序列，X最多只有包含其中的前n-1个字符
    num_examples = (len(corpus_indices) - 1) // num_steps  # 下取整，得到不重叠情况下的样本个数
    # 得到可取到每个时间步数的字符组的下标
    example_indices = [i * batch_size for i in range(num_examples)]
    random.shuffle(example_indices)
    
    def _data(i):
        return corpus_indices[i: i+num_steps]
    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        
    for i in range(0, num_examples, batch_size):
        # 每次选出batch_size个样本
        batch_size_indices = example_indices[i:i+batch_size]
        X = [_data(j) for j in batch_size_indices]
        Y = [_data(j+1) for j in batch_size_indices]
        yield torch.tensor(X, device=device), torch.tensor(Y, device=device)
        

In [30]:
my_seq = list(range(30))
for X, Y in data_iter_random(my_seq, batch_size=2, num_steps=6):
    print('X: ', X, '\nY:', Y, '\n')

X:  tensor([[ 6,  7,  8,  9, 10, 11],
        [ 2,  3,  4,  5,  6,  7]]) 
Y: tensor([[ 7,  8,  9, 10, 11, 12],
        [ 3,  4,  5,  6,  7,  8]]) 

X:  tensor([[4, 5, 6, 7, 8, 9],
        [0, 1, 2, 3, 4, 5]]) 
Y: tensor([[ 5,  6,  7,  8,  9, 10],
        [ 1,  2,  3,  4,  5,  6]]) 



## 相邻采样
在相邻采样中，相邻的两个随机小批量在原始序列上的位置相毗邻。**即当前batch是上一个batch是相邻样本，先切分，后按顺序采样。**

In [31]:
def data_iter_consecutive(corpus_indices, batch_size, num_steps, device=None):
    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    corpus_len = len(corpus_indices) // batch_size * batch_size
    corpus_indices = corpus_indices[:corpus_len]
    indices = torch.tensor(corpus_indices, device=device)
    # resize成batch_size行
    indices = indices.view(batch_size, -1)
    batch_num = (indices.shape[1] - 1) // num_steps
    for i in range(batch_num):
        i = i * num_steps
        # 行全取，每次取num_steps列
        X = indices[:,i:i+num_steps]
        Y = indices[:,i+1:i+num_steps+1]
        yield X, Y

# 循环神经网络

## 循环神经网络的构造
**我们先看循环神经网络的具体构造。**
	
	i.首先网络同样是有三层神经网络构成，输入是n个时态的输入值记为Xt。
	ii.其次是隐藏层，隐藏层的输出记为Ht。
	iii.最后是输出层，输出层是不同时间位置对应的Ot

**RNN的前向传播**
用公式来表示一个最简单的循环神经网络前向传播的方法如下：
$$
\boldsymbol{H}_t = \phi(\boldsymbol{X}_t \boldsymbol{W}_{xh} + \boldsymbol{H}_{t-1} \boldsymbol{W}_{hh}  + \boldsymbol{b}_h).
$$
$$
\boldsymbol{O}_t = \boldsymbol{H}_t \boldsymbol{W}_{hq} + \boldsymbol{b}_q.
$$
以上公式表示的是循环神经网络的前向传播的公式。

**RNN的反向传播**
对于循环神经网络来说，其求取反向传播的过程与正常的DNN有一些区别。
具体求导的过程可以参考[知乎连接RNN求导方式](https://zhuanlan.zhihu.com/p/32930648)
RNN最终的损失是所有时刻的损失的和。
RNN在反向传播的过程的时候，某时刻t的梯度和前面所有时刻的梯度都有关。

**RNN反向传播的过程中为什么容易出现梯度爆炸或是梯度消失**
这是RNN最大的缺陷，由于这一缺陷，使得RNN在长文本中难以训练。
梯度消失指的是在反向传播梯度的时候，出现了梯度消失的现象，梯度向后传播的时候越乘越小，使得权重无法更新，导致训练失败。而梯度爆炸所带来的的问题就是梯度过大，梯度在向前传播的过程中，越变越大，从而大幅度更新网络参数，造成网络不稳定。在极端情况下，权重的值变得特别大，一直与结果会溢出。	
总之，无论是梯度消失还是梯度爆炸，都是源于网络结构太深，造成网络权重不稳定，从本质上来讲是
因为梯度反向传播中的**连乘效应**。当句子特别长的时候，也就是状态很多的时候，梯度需要连乘的项就越来越多。而对于使用tanh作为激活激活函数的RNN。每次本次状态对前一次状态求导实际上都是进行一次tanh求导，当Wx/或者Ws来说，当这个值本身就很小时，梯度就会越来越小，从而出现梯度消失。而当Wx或者Ws很大的时候，梯度将会越乘越大，出现梯度爆炸的情况。




## 从零开始实现RNN

In [1]:
import torch
import torch.nn as nn
import time
import math
import sys
sys.path.append("/home/kesci/input")
import d2l_jay9460 as d2l
# 歌词字符集、字符索引映射列表、索引字符映射列表、歌词词库大小
(corpus_indices, char_to_idx, idx_to_char, vocab_size) = d2l.load_data_jay_lyrics()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
# one_hot 编码， 将词库中的词索引转换为无数量比较的one_hot编码
def one_hot(x, n_class, dtype=torch.float32):
    """
        x:输入的词组对应的数字索引（确切的说是连续的词组）
        n_class：词库的数量
    """
    result = torch.zeros(x.shape[0], n_class, dtype=dtype, device=x.device)
    # scatter_(input, dim, index, src) → Tensor
    # 将src中的所有值按照index确定的索引写入本tensor中。
    # 其中索引是根据给定的dimension，dim按照gather()描述的规则来确定。
    result.scatter_(1, x.long().view(-1, 1), 1)
    return result

x = torch.tensor([0, 2])
x_tensor_hot = one_hot(x, vocab_size)
print(x_tensor_hot)
print(x_tensor_hot.shape)
print(x_tensor_hot.sum(axis=1))
    

tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.]])
torch.Size([2, 1027])
tensor([1., 1.])


In [7]:
# 每次采样的小批量的形状是（批量大小， 时间步数）。
# 下面的函数将这样的小批量变换成数个形状为（批量大小，词典大小）的矩阵，矩阵个数等于时间步数。
# 输入的X行代表的意义是batch_size的数量，X列代表的含义是index个时刻
# 每次取出batch_size个T时刻的词对应的词向量。
def to_onehot(X, n_class):
    return [one_hot(X[:, i], n_class) for i in range(X.shape[1])]

X = torch.arange(10).view(2, 5)
inputs = to_onehot(X, vocab_size)
print(len(inputs), inputs[0].shape)

5 torch.Size([2, 1027])


# 初始化模型参数

In [8]:
# 首先确定网络结构
# 三层结构，参数为Wx,Ws,W0,b1,b2，分别代表输入值隐藏层，前一隐藏状态至当前隐藏状态，隐藏层到输出层
def get_params():
    def _one(shape):
        param = torch.zeros(shape, device=device, dtype=torch.float32)
        nn.init.normal_(param, 0, 0.01)
        return torch.nn.Parameter(param)
    # 隐藏层参数
    W_xh = _one((num_inputs, num_hiddens))
    W_hh = _one((num_hiddens, num_hiddens))
    b_h = torch.nn.Parameter(torch.zeros(niddens, device=device))
    # 输出层参数
    W_hq = _one((num_hiddens, num_outputs))
    b_q = torch.nn.Parameter(torch.zeros(num_outputs, device=device))
    
    return W_xh, W_hh, b_h, W_hq, b_q

## 定义模型
函数rnn用循环的方式一次完成循环神经网络每个时间步的计算

In [10]:
def rnn(inputs, state, params):
    # inputs和outputs皆为num_steps个形状为(batch_size, vocab_size)的矩阵
    W_xh, W_hh, b_h, W_hq, b_q = params
    H, = state
    outputs = []
    # inputs是不同的时态*（batch_size,vocab_size）list
    for X in inputs:
        H = torch.tanh(torch.matul(X, W_xh) + torch.matmul(H, W_hh) + b_h)
        Y = torch.matmul(H, W_hq) + b_q
        outputs.append(Y)
    return outputs, (H,)

In [11]:
def init_rnn_state(batch_size, num_hiddens, device):
    return (torch.zeros((batch_size, num_hiddens), device=device), )

# TODO
最近导师开始催促完成项目
之后再补上